In [ ]:
from INFPHY201 import *

# Lecture 3: Transformations of quantum states
 (2020) 

Welcome to Lecture 3!<br>
After this lecture you will be able to:
1. Explain the exponential representation of unitary transformations
2. Interpret the meaning and the action of unitary transformations
3. Model the transformation of a qubit
4. Model the time evolution of a quantum state

---
***Recap of Lecture 2:***
_In Lecture 2 we saw how to model Measurements (the second pillar of QM). We think of a measurement as an orthonormal basis in the Hilbert space of the system, where each vector in the basis corresponds to a possible outcome. The actual outcome that we obtain is random, and it follows the probability distribution given by the absolute value squared of the coefficients of the state vector written in this basis. The measurement collapses the state vector of the system onto the basis element corresponding to the result, and no trace of the previous state is left. Given the correspondence beween bases and measurements, we use Hermitian matrices to represent Observable quantities. The idea is to use the eigenbasis of the matrix as the basis for the measurement of that Observable. Then we could think of the eigenvalues as simply the numerical quantities associated with the outcomes (which allows us for examples to include measurement units). Finally, we associated each element of the basis to a projector onto that eigenspace. This allows us to treat projectors as "probability operators", as their expectation value is the probability of obtainin that outcome._

---
***Preview of Lecture 3:***
_Today we model transformations of quantum states with Unitary operators. The reason is that Unitary operators don't change the length of complex vectors: they simply rotate them into other vectors of the same length, i.e. they transform states into states. Our goal today is to express unitary transformations in a clever and useful way, and to include the transformation of a qubit and the time-evolution as particular, but very important cases._

---

## 1. Unitary operators as matrix exponentials

The idea here is to have a _parametrized_ unitary $U(\epsilon)$, where the parameter $\epsilon$ regulates the "amount of transformation", so that if $\epsilon=0$ we have the identity matrix $U(0) = \mathbb{1}$. Not only that, but our clever definitions make transformations into a group: $U(\epsilon_2)U(\epsilon_1) = U(\epsilon_1+\epsilon_2)$ (obviously they have to be transformations of the same kind for this to hold!). So let's see how we can make this happen.

### 1.1 Exponential form of a 1-parameter unitary
The formula that we are going to use is in terms of the matrix exponential:

$$
U(\epsilon) = \exp(i\epsilon M)
$$

where $M$ is some Hermitian matrix, called the "_generator_" of the transformation (we will learn a great deal about generators). This is a simple way of parametrizing a unitary transformation. Don't be misguided by the similarity with the complex phase $e^{i\epsilon}$: although the matrix exponential is related to complex phases, $\epsilon$ should not be thought of as an angle. It is if the generator has integer eigenvalues, but it's not an angle in general, which means that in general $U(2\pi)\neq\mathbb{1}$ or even that $U(\epsilon)=\mathbb{1}$ only for $\epsilon = 0$.

We can justify the exponential formula by looking at the spectrum: the spectrum of $M$ is made of real numbers, which implies that the spectrum of $\exp(i\epsilon M)$ (for real $\epsilon$), must be on the unit circle of the complex plane, just like the spectrum of a unitary matrix.

### 1.2 Computing a 1-parameter unitary
How do we compute the exponential of a matrix? A way of thinking about it is via the Taylor series of the exponential:

$$
\exp(A) = \sum_{k=0}^\infty \frac{A^k}{k!}
$$

In this expression, $A^k$ is the matrix $A$ multiplied by itself $k$ times ($A\cdot A\cdot\dots\cdot A$), which is NOT the matrix obtained by raising each matrix element to the $k$-th power! This implies that $\exp(A)$ is not obtained by taking the exponential of each matrix element. Be careful with this, because if you write `np.exp(A)`, numpy will compute the exponential of each element, which is NOT what we want! The function we want is `scipy.linalg.expm`.

Although the Taylor series is a mathematically legitimate way of representing the matrix exponential, it's not computationally sound because we would need a loooot of terms before the series converges. The easiest approach is to exploit the fact that for a diagonal matrix $D$ it is true that $\exp(D)$ is simply the exponential of the elements on the diagonal. So if we diagonalize $A$ we can write 

$$
\exp(A) = \exp(V^\dagger D V) = \sum_{k=0}^\infty \frac{(V^\dagger D V)^k}{k!} = \sum_{k=0}^\infty \frac{V^\dagger D^k V}{k!} = V^\dagger\sum_{k=0}^\infty \frac{D^k}{k!}V = V^\dagger\exp(D)V
$$

---
#### Activity 1: Matrix Exponentials (10 minutes)
Let's use the same Hermitian matrix that we used in lecture 2:
$$
M = 
\begin{pmatrix}
1 & 2 & 3 & 4\\
2 & 0 & 2i & 5\\
3 & -2i & -1 & 0\\
4 & 5 & 0 & 0
\end{pmatrix}
$$

1. Verify that the formula $V^\dagger\exp(D)V$ gives the same result as `scipy.linalg.expm`. Use `numpy.linalg.eig` to obtain the eigendecomposition (it returns the diagonal of $D$ and $V^\dagger$).
---

In [ ]:
M = np.array([
    [1,2,3,4],
    [2,0,2j,5],
    [3,-2j,-1,0],
    [4,5,0,0]
])

exp_M = expm(M)

In [ ]:
from numpy.linalg import eig

In [ ]:
d, Vdagger = eig(M)

In [ ]:
VdV = Vdagger@np.diag(np.exp(d))@np.conj(Vdagger.T)

In [ ]:
np.linalg.norm(exp_M - VdV)

In [ ]:
np.allclose(exp_M, VdV)

## 2. What does the transformation do?
Okay, we have defined it. But what does $U(\epsilon)$ actually do? Can we "picture" it? It turns out that there is a clever way to think about it and (perhaps unsurprisingly) it has to do with the eigenbasis of the generator (I hope you're beginning to spot a pattern here!).

### 2.1 Interpretation of $U(\epsilon) = \exp(i\epsilon M)$
From the formula $U(\epsilon) = V^\dagger\exp(D)V$ (where $i\epsilon M = V^\dagger D V$), we can deduce that the action of $U(\epsilon)$ is to vary the phase of the eigenvectors of $M$! Another way of expressing $U(\epsilon)$ is in fact:

$$
U(\epsilon)=V^\dagger
\begin{pmatrix}
\exp(i\epsilon\lambda_1) & 0 & 0 & \cdots\\
0 & \exp(i\epsilon\lambda_2) & 0 & \cdots\\
0 & 0 & \exp(i\epsilon\lambda_3) & \cdots\\
\vdots & \vdots & \vdots & \ddots\\
\end{pmatrix}
V
$$

which means that the action of $U(\epsilon)$ rotates the basis into the eigenbasis of $M$ (that's what $V$ does), then it rotates the phase of the eigenvectors of $M$ and finally it rotates the basis back to where it was (that's what $V^\dagger$ does). So the action on an element $|v_i\rangle$ of the eigenbasis of $M$ is simply to rotate its phase by an angle that depends on the eigenvalue $\lambda_i$ and on $\epsilon$:

$$
U(\epsilon)|v_i\rangle = \exp(i\epsilon\lambda_i)|v_i\rangle
$$

This is the secret to understand unitary transformations: if we express $U(\epsilon)$ in the eigenbasis of its generator $M$ we obtain:

$$
U(\epsilon) = \sum_k e^{i\epsilon\lambda_k}|v_k\rangle\langle v_k|
$$

Notice that on the diagonal we have phases that rotate at different "speeds", because while $\epsilon$ is a value that they share, they have different eigenvalues (modulo multiplicity).

So the simplest way of visualizing the action of a unitary transformation on a quantum state is to think of the quantum state in the basis of the generator of the transformation! Then the action of $U(\epsilon)$ is simply to add a phase to each eigenstate! Let's see an example. Consider a state $|\psi\rangle$ and the unitary transformation $U(\epsilon) = \exp(i\epsilon M)$. Then, we simply have to express $|\psi\rangle$ in the eigenbasis of $M$:

$$|\psi\rangle = \sum_k c_k|v_k\rangle$$

and the action of $U(\epsilon)$ is:

$$U(\epsilon)|\psi\rangle = \sum_k c_ke^{i\epsilon \lambda_k}|v_k\rangle$$

NOTE: _If any unitary quantum evolution works like this, what is the big deal with quantum computing? A quantum algorithm is a quantum circuit, which is a unitary evolution. Why does it give us so much computational power? The answer is in the size of the Hilbert space: the dimension of a state vector of $n$ qubits is $2^n$, so the size of the unitary transformation representing a quantum circuit is $2^n \times 2^n$, which for large $n$ (e.g. already for a few tens of qubits) is too large to diagonalize on classical hardware. So it's impossible to simulate. So if we use **it** to compute stuff, it will be more powerful than classical hardware._

---
#### Activity 2: Periods (10 minutes, pen and paper!)
1. Consider the matrix 
$$
M = 
\begin{pmatrix}
2/3 & 0 \\
0 & 2
\end{pmatrix}
$$
Let's find the period of $U(\epsilon) = \exp(i\epsilon M)$: what is the smallest value $\epsilon > 0$ such that $\exp(i\epsilon M)= \mathbb{1}$?
2. Can you think of a value $\lambda$ such that the matrix
$$
M = 
\begin{pmatrix}
1 & 0 \\
0 & \lambda
\end{pmatrix}
$$
has infinite period?

NOTE: _If a transformation happens to be periodic with period $T$, we can choose to reparametrize the transformation $\epsilon\rightarrow\phi \frac{T}{2\pi}$. Now $U(\phi)$ has period $2\pi$, so we can think of $\phi$ as an actual angle._

### 2.2 Interpretation of the generator $M$ (and a bit about units)

The generator $M$ is a Hermitian matrix. But we already have an interpretation of Hermitian matrices, they are Observables! Their eigenbasis determines a measurement basis and so on... Well, this interpretation carries over, i.e. Observables are generators of transformations and generators of transformations are Observables. So given an Observable, what transformation does it generate?

**Answer in the chat:** _If we measure $M$ on the state $|\psi\rangle$ and the state $\exp(i\epsilon M)|\psi\rangle$ do we observe different outcome probabilities?_

Now let's ask the reverse question: given a transformation, can we find the meaning of its generator? Sometimes this connection is extremely clear, sometimes a bit less so. The general rule is that the quantity in the exponential must be _dimensionless_ otherwise it wouldn't make sense (what would be the meaning of its Taylor series?). So at least we can say that whatever units of measure the eigenvalues of $M$ have, they must be conjugated to the units of $\epsilon$, i.e. their product must be dimensionless.

There is a little thing that we must mention at this point (which is not _that_ important, but we must not forget it): we have been ignoring the presence of the inverse of Planck's constant $\hbar^{-1}$ at the exponent because usually we work in "natural units" where $\hbar = 1$. This is a convenient simplification, so what is left is to rememeber that the units of $\hbar$ are [Energy$\cdot$Time]. So for example, if $M$ is the observable of Energy and $\epsilon$ is in units of Time, then $\epsilon M/\hbar$ is dimensionless, which is what we want:

$$
\exp\bigl(i\underbrace{\frac{\epsilon}{\hbar}}_{\mathrm{inverse}\atop \mathrm{units\ of} M}M\bigr)
$$

This is not too different from something you should be familiar with from Fourier analysis: in the Fourier kernel $e^{i\omega t}$ the frequency $\omega$ is in units of [Time$^{-1}$] so that the product $\omega\, t$ is dimensionless as it should be. Now we also have a factor $\hbar^{-1}$, which cancels out a unit of Time and introduces one of Energy. This is just a rescaling of the Fourier relation between two quantities:

$$
\begin{align}
\left[\frac{t}{\hbar}H\right] &= \frac{[\mathrm{Time}]}{[\mathrm{Energy}\cdot\mathrm{Time}]}[\mathrm{Energy}]
\end{align}
$$

Wait a second! If $M$ is the observable of Energy, that means $M$ is the Hamiltonian. So if $M$ is the Hamiltonian then $U(t)$ (let's write $\epsilon$ as $t$ since it's in units of Time) is a transformation that makes time evolve!

Other example: the observable $X$ of position is in units of [Length]. So in the expression $U(\epsilon) = \exp(i\epsilon X)$, $\epsilon$ must be in units of [Length$^{-1}\cdot$Energy$\cdot$Time] (the last two are there to cancel out the units of $\hbar^{-1}$). Hmm, this doesn't look familiar yet (but it is, I promise). Let's express [Energy] in a more convenient way: [Energy] = [Length$\cdot$Force] = [Length$\cdot$Mass$\cdot$Acceleration] = [Length$^{2}\cdot$Mass$\cdot$Time$^{-2}$], putting everything together we obtain that $\epsilon$ is in units of [Mass$\cdot$Length$\cdot$Time$^{-1}$]. This is a mass multiplied by a velocity! This is _momentum_. So position and moementum in quantum mechanics are conjugated to each other!

$$
U(x) = \exp(i x P)\\
U(p) = \exp(i p X)
$$

We will see that these equations mean that the position Observable $X$ generates shifts in momentum (a particle gets a "kick") and the momentum Observable generates shifts in position.

## 3. Qubit transformations

We now apply these ideas to qubits, i.e. the states are 2-dimensional and all the matrices (Hermitian, Unitary etc..) are $2\times 2$. Again, we will see that thanks to the small dimensionality of the system, we can solve everything analytically.

### 3.1 Pauli operators and qubit rotations

In Lecture 1 you experimented with qubit states and modified them "manually" to make them rotate around the Bloch sphere. Now we will construct the unitary transformations that can do that for you. In any case, we expect these unitary transformations to be in the form $U(\epsilon) = \exp(i\epsilon M)$ for some $2\times 2$ matrix $M$, so the real question is: what is $M$? Let's start from the most intuitive one: if we want to induce a _phase_ difference between the two canonical basis states $|0\rangle$ and $|1\rangle$, what should we do? An example could be to phase-shift $|0\rangle$ in one direction, i.e. $e^{i\epsilon}|0\rangle$ and phase shift $|1\rangle$ in the opposite direction, i.e. $e^{-i\epsilon}|1\rangle$.

Since $|0\rangle$ and $|1\rangle$ are the basis elements of the canonical basis, we can do this with a diagonal matrix:

$$U(\epsilon)=\begin{pmatrix}e^{i\epsilon}&0\\0&e^{-i\epsilon}\end{pmatrix}= e^{i\frac{\phi}{2}}|0\rangle\langle 0| + e^{-i\frac{\phi}{2}}|1\rangle\langle 1|$$

This is given in exponential form $\exp(i\epsilon \sigma_z)$ by defining 

$$\sigma_z=\begin{pmatrix}1&0\\0&-1\end{pmatrix}$$

The reason why we called the matrix $\sigma_z$ will be clear in a moment. The transformation $U(\epsilon)$ is clearly periodic with period $\epsilon = \pi$, so in order to use an actual angle we define the variable $\phi = 2\epsilon$ and reparametrize:

$$
U_z(\phi) = \exp\left(i\frac{\phi}{2}\sigma_z\right) 
$$

The reason why we are using the little $z$ is because when we apply the transformation $U_z(\phi)$ to a qubit state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$, we rotate it around the $z$ axis of the Bloch sphere by an angle $\phi$! Notice that the $z$ axis connects the North pole of the Block sphere to the South pole, i.e. the states $|0\rangle$ and $|1\rangle$. This is not a coincidence: if we use the basis $\{|v_0\rangle, |v_1\rangle\}$ to define a transformation :

$$U(\phi) = e^{i\frac{\phi}{2}}|v_0\rangle\langle v_0| + e^{-i\frac{\phi}{2}}|v_1\rangle\langle v_1|$$

such transformation rotates the qubit around the axis defined by the basis $\{|v_0\rangle, |v_1\rangle\}$ by an angle $\phi$ (recall that orthogonal states are _opposite points_ on the Bloch sphere, so they always define an axis). As examples of alternative rotations, we can write the rotations around the $x$ axis and around the $y$ axis. For these, we need the basis that identifies the two opposite points on the $x$ axis: $\{|+\rangle, |-\rangle\}$ where $|\pm\rangle = (|0\rangle\pm|1\rangle)/\sqrt{2}$ and the basis that identifies the two opposite points on the $y$ axis: $\{|R\rangle, |L\rangle\}$ where $|R/L\rangle = (|0\rangle\pm i|1\rangle)/\sqrt{2}$.
The generators of the rotations around the $x$ and $y$ axes, expressed in the canonical basis are:

$$
\sigma_x = \begin{pmatrix}0&1\\1&0\end{pmatrix}\\
\sigma_y = \begin{pmatrix}0&-i\\i&0\end{pmatrix}
$$

which together with $\sigma_z$ are the famous "_Pauli matrices_". So the Pauli matrices are the generators of rotations around the axes of the Bloch sphere. Any unitary qubit transformation can be decomposed into rotations around the $x$, $y$, $z$ axes. More interestingly, for any unitary qubit transformation there exists an axis (i.e. a basis) which implements it directly (this is related to the group theory of $SU(2)$ and $SO(3)$).

---
#### Activity 3: qubit rotations (15 min, pen and paper!)
Using the definitions of the bases and the formula $U(\phi) = e^{i\frac{\phi}{2}}|v_0\rangle\langle v_0| + e^{-i\frac{\phi}{2}}|v_1\rangle\langle v_1|$, write the transformations $U_x(\phi)$ and $U_y(\phi)$ explicitly in matrix form.

---
#### Activity 4: qubit rotations (20 min, coding!)
1. Write 3 functions that take an angle $\phi$ and return the three unitary transformations $U_x(\phi)$, $U_y(\phi)$ and $U_z(\phi)$. The signature should be `f(float)->np.array(complex)`
2. Test the functions using a `qutip.Bloch()` sphere object: start with the state $|0\rangle$, rotate it by $\pi/2$ around the $y$ axis, then by $-\pi/2$ around the $z$ axis and then by $-\pi/2$ around the $x$ axis. You should find yourself back up on the North pole. Visualize these transformations on the Bloch sphere by making little steps.

NOTE: _The Pauli operators are conjugate to the angle $\phi$, so they are Observables of **angular momentum**._

In [ ]:
def Ux(phi):
    Jx = np.array([[0.0,1.0],[1.0,0.0]])
    return expm(1j*phi/2*Jx)

def Uy(phi):
    Jy = np.array([[0.0,-1j],[1j,0.0]])
    return expm(1j*phi/2*Jy)

def Uz(phi):
    Jz = np.array([[1.0,0.0],[0.0,-1.0]])
    return expm(1j*phi/2*Jz)

In [ ]:
sphere = Bloch()

In [ ]:
north_pole = np.array([1.0,0.0])

In [ ]:
sphere.clear()
path1 = [Qobj(Uy(phi)@north_pole) for phi in np.linspace(0,-np.pi/2,10)]
sphere.add_states(path1)

path2 = [Qobj(Uz(phi)@(Uy(-np.pi/2)@north_pole)) for phi in np.linspace(0,-np.pi/2,10)]
sphere.add_states(path2)

path3 = [Qobj(Ux(phi)@(Uz(-np.pi/2)@Uy(-np.pi/2)@north_pole)) for phi in np.linspace(0,-np.pi/2,10)]
sphere.add_states(path3)

In [ ]:
sphere.show()

## 4. Time evolution
We now turn our attention to larger systems and to time evolution. Remember last week, when we diagonalized the Hamiltonian of a particle in a box? Today we will see where that comes from, as well as a bit on the Schrödinger equation.

### 4.1 How to write a Hamiltonian
Writing a Hamiltonian is an important thing to know. Once we have the Hamiltonian, we can compute the time evolution operator and with that we can fast-forward time and rewind the clock on any quantum state!

We need to begin from the role of the Hamiltonian as the Observable of energy. Energy in mechanical systems (like a particle in a box) comes in two forms: kinetic and potential: $H = H_\mathrm{kinetic} - H_\mathrm{potential}$. Classically, the kinetic energy can be written as

$$
E_\mathrm{kinetic} = \frac{P^2}{2m}
$$

Where $P$ is the momentum and $m$ is the mass of the particle (which is just a numerical parameter). So in order to write the kinetic contribution to the Hamiltonian, we need the momentum Observable. What is the momentum observable? To answer this question we need to go back for a second to the fact that position and momentum are continuous quantities, and their Observable are actually defined in an infinite-dimensional Hilbert space. This is very convenient because in an infinite-dimensional Hilbert space we can do calculus (integrals, derivatives and so on). In fact, those are our only tools to define operators (which in that case cannot be matrices, because the space is infinite-dimensional).

So what we'll do is we will find the momentum Observable $P$ with a trick. First of all, we'll need something that contains $P$. Luckily we know that the unitary operator that shifts the position of a particle is generated by the momentum Observable:

$$
U(x) = \exp(i x P)
$$

So we can write:

$$
-i\frac{\partial}{\partial x}U(x) = P\exp(ixP) = PU(x)
$$

Which means that:

$$
-i\frac{\partial}{\partial x} = P
$$

and therefore 
$$
P^2 = -\frac{\partial^2}{\partial x^2}
$$

This makes perfect sense in an infinite-dimensional vector space, because the operators on it are in differential form (they are not matrices). But what happens when we discretize the space? We saw that integrals become sums, what about derivatives? This is simpler than one may think: a derivative is the limit for $\delta x\rightarrow 0$ of $$\frac{\psi(x + \delta x) - \psi(x)}{\delta x}$$ In a discrete space though, we can't have $\delta x$ go all the way to zero! It will have to shrink at most to the size of the smallest interval. But then $\psi(x+\delta x)$ and $\psi(x)$ are two amplitudes that are next to each other in the discrete space. So that tells us how to construct the matrix the implements the discrete derivative: 

$$
\frac{\partial}{\partial x} = 
\begin{pmatrix}
-1 & 1 & 0 & 0 & \cdots\\
0 & -1 & 1 & 0 &\cdots\\
0 & 0 & -1 & 1 & \cdots\\
\vdots & \vdots & \vdots& \ddots & \ddots
\end{pmatrix}
$$

This is a well-known technique in finite element analysis. Okay, we are almost done! To obtain the second derivative we can apply the formula for the (symmetric) second derivative $\frac{1}{\delta x^2}(\psi(x-\delta x) - 2\psi(x) + \psi(x+\delta x))$ and we obtain the matrix that implements it:

$$
\frac{\partial^2}{\partial x^2} = 
\begin{pmatrix}
-2 & 1 & 0 & 0 & \cdots\\
1 & -2 & 1 & 0 &\cdots\\
0 & 1 & -2 & 1 & \cdots\\
0 & 0 & 1 & -2 & \ddots\\
\vdots & \vdots & \vdots& \ddots & \ddots
\end{pmatrix}
$$

With this, we can finally build the symmetric Hamiltonian:

$$
H = \frac{1}{2m}
\begin{pmatrix}
2 & -1 & 0 & 0 & \cdots\\
-1 & 2 & -1 & 0 &\cdots\\
0 & -1 & 2 & -1 & \cdots\\
0 & 0 & -1 & 2 & \ddots\\
\vdots & \vdots & \ddots& \ddots & \ddots
\end{pmatrix}
$$

And that tells us how to construct the kinetic term of the Hamiltonian! The potential term is easy (especially in our case). If we have a potential that depends on the position of the particle (like a potential well, or a harmonic oscillator like a mass on a spring etc...) then it will be diagonal in the position basis. It will simply be a diagonal matrix with the value of the potential $V(x)$ at the discrete position $x$. A box with solid boundaries has zero potential inside and infinite potential outside. So for us it's trivial to implement the walls of the box: they correspond to the first and last index! Our quantum state is guaranteed to stay in the box because there are no indices to describe positions outside of it! So here's our Hamiltonian:

Finally, because diagonalization algorithms can get picky when things are not symmetric, we will shift

---
#### Activity 4: Implement the Hamiltonian (10 minutes)
Write a function that takes the mass of the particle and the dimension of the Hilbert space and returns the matrix of the Hamiltonian. The signature should be `f(float, int) -> np.array(complex)`. TIP: the function `np.diag()` has a second argument that you can use to fill the diagonal of your choice, not just the central one.

---

In [ ]:
def H(m, dim):
    P_squared = np.diag(2*np.ones(dim), k=0) + np.diag(-1*np.ones(dim-1), k=1) + np.diag(-1*np.ones(dim-1), k=-1)
    return P_squared/(2*m)

### 4.2 How to compute the action of the time evolution operator
We are going to see two ways of computing time evolution.

The first way to compute the action of $U(t)$ on a state is to exponentiate the Hamiltonian for a fixed choice of time interval $\Delta t$, and the resulting matrix advances time by $\Delta t$ every time we use it to multiply the state:

$$
\begin{align}
U(\Delta t)|\psi(0)\rangle &= |\psi(\Delta t)\rangle\\
U(\Delta t)|\psi(\Delta t)\rangle &= |\psi(2\Delta t)\rangle\\
\mathrm{etc}\dots
\end{align}
$$

So we need to compute $U(\Delta t)$ only once and for all.

If we want more flexibility, or for example to have $|\psi(t)\rangle$ for all times $t$ without having to recompute a new matrix exponential for all $t$, we can adopt the second way. We express $|\psi(0)\rangle$ in the eigenbasis of $H$ and then we multiply the $k$-th amplitude by $\exp(it\lambda_k)$. This is a much simpler calculation than a whole matrix exponential (so it's much faster) and it gives us $|\psi(t)\rangle$ directly.

Let's implement both. We start by computing $U(\Delta t)$ for a small time interval, say $\Delta t=1$ (this is a small interval in relation to the energy):

In [ ]:
U = expm(1j*1.0*H(0.5, 200))

In [ ]:
psi = np.array([np.exp(-(x-100)**2/(2*25)) for x in range(200)])
psi = psi/np.linalg.norm(psi)

In [ ]:
speed = 100
state = psi
for k in range(100):
    plt.plot(abs(state)**2)
    state = U@state

I've written a function (very inefficient as it creates a new figure object each time) to create a "live plot". If you find a way to make it faster (for instance by reusing the same figure object) let me know!

In [ ]:
from collections import defaultdict
data = defaultdict(list)

psi = np.array([np.exp(-(x-100)**2/(2*25)) for x in range(200)])
speed = 100
state = psi*np.exp(-1j*speed*np.linspace(-1,1,200))

for i in range(100):
    state = U@state
    data['prob']= np.abs(state)**2
    live_plot(data)

Now we look at the second method:

In [ ]:
eigenvalues, Vdagger = np.linalg.eig(H(0.5, 200))

psi = np.array([np.exp(-(x-100)**2/(2*25)) for x in range(200)])

# the wave function in the eigenbasis of H
psi_H = np.conj(Vdagger.T)@psi

In [ ]:
def psi_evolved(t):
    return Vdagger@(np.exp(1j*eigenvalues*t)*psi_H)

In [ ]:
data = defaultdict(list)

for t in range(100):
    data['prob']= np.abs(psi_evolved(t))**2
    live_plot(data)